# Preprocessing for User Run 23/03 Sensor Data

### Imports

In [162]:
import os
import pandas as pd

### Function to average on a minute base where possible

In [163]:
def average_per_min(df):
    n_col = len(df.columns)
    df['_time'] = pd.to_datetime(df['_time'], utc=True)
    df['Date'] = df['_time'].dt.date
    df['Hour'] = df['_time'].dt.hour
    df['Minute'] = df['_time'].dt.minute
    df.drop(['_time'], axis = 1, inplace=True)
    
    dfn = df.groupby(['Date', 'Hour', 'Minute'])[df.columns[0:n_col-1]].mean()
    dfn = dfn.reset_index()
    dfn['_time'] = dfn['Date'].astype(str) + ' ' + dfn['Hour'].astype(str) + ':' + dfn['Minute'].astype(str) + ':00'
    dfn['_time'] = pd.to_datetime(dfn['_time'], format="%Y-%m-%d %H:%M:%S")
    dfn.drop(['Date', 'Hour', 'Minute'], axis = 1, inplace=True)
    return dfn

### Funtion to read a file ffill and bfill and apply the averaging

In [164]:
def read_sensors_and_process(path, df):
    file = pd.read_csv(path)
    file['_time'] = file['_time'].apply(lambda x: x.split('.')[0])
    file['_time'] = pd.to_datetime(file['_time'], format="%Y-%m-%d %H:%M:%S")
    file = file.ffill()
    file = file.bfill()
    filen = average_per_min(file)
    df = pd.merge(df, filen, on='_time', how='left')
    return df

### Getting files from user run

In [165]:
files = [file for file in os.listdir('../../Sensor Data/2023_3/') if not file.startswith('.')]

### Setting up target dataframe

In [166]:
startTime = pd.to_datetime('18/09/2023, 08:30:00', format="%d/%m/%Y, %H:%M:%S")
endTime = pd.to_datetime('20/10/2023, 08:30:00', format="%d/%m/%Y, %H:%M:%S")
targetDF = pd.DataFrame()
targetDF['_time'] = pd.date_range(start=startTime, end=endTime, freq='1min')

### Going through each file and applying the previous functions

In [167]:
for f in files:
    
    targetDF = read_sensors_and_process('../../Sensor Data/2023_3/' + f, targetDF)
targetDF = targetDF.ffill()
targetDF = targetDF.bfill()

In [168]:
targetDF['_time'] = targetDF['_time'].dt.strftime("%d/%m/%Y, %H:%M:%S")

In [169]:
targetDF.dropna(axis=1, how='all', inplace=True)

In [170]:
targetDF.rename(columns={'_time': 'Time'}, inplace=True)

### Saving

In [171]:
targetDF.to_csv('../../Processed Sensors/2023_3.csv', index=False)